## Constants

In [1]:
import numpy as np
from scipy.constants import hbar,h,e,c
F0=2.0678e-15

In [2]:
e_eff=(11.45+1)/2## for all calculations

In [3]:
Mhz=(2*np.pi*1e6)## divide on it to obtain value in mhz

## Resonator properties

$$\kappa = \frac{\omega_r}{2Q}=\frac{\pi f_r}{Q}$$ in radians per second and the envelope of oscillation decays proportional to $$e^{-\kappa t} $$  for energy multiply by two ( in comparison with Koch paper, i found kappa should be calculated for energy relaxation)


In [4]:
fr=6.5e9
Q=6000
k=2*np.pi*fr/Q 
1/k*1e9 , k/Mhz

(146.91225516174953, 1.0833333333333335)

### frequency shift due to end capacitance

In [5]:
# Cclaw=52e-3#pF
# Cclaw=7.3416e-3*e_eff#pF
Cclaw=7.643e-3*e_eff#pF
Cl_r=156.942#pF/m
Leff_add=Cclaw/Cl_r
Cclaw*1e3,Leff_add*1e3 ## add it to open end

(47.57767499999999, 0.3031545093091715)

### coupling between resonator and qubit

In [6]:
fq=4.1e9
Zr=56.3
# Ccr=0.3656e-15*e_eff
# Cq=18.56e-15*e_eff
Ccr=0.3645e-15*e_eff
Cq=20.23e-15*e_eff
Cr=1/(2*np.pi*fr*Zr)
g=2*np.pi*Ccr/np.sqrt(Cr*Cq)*np.sqrt(fr*fq)
Ccr*1e15,Cq*1e15,g/Mhz

(2.2690125, 125.93175, 50.051683050895036)

### dispersion shift

In [7]:
fq=4.1e9
hi=g**2/(2*np.pi*(fr-fq))
hi/Mhz

1.043821240094689

### Purcell rate

In [8]:
T=(1/k)*(2*np.pi*(fq-fr)/g)**2
T*1e6

337.78716014268315

## Qubit Frequencies

In [9]:
def I(f,Ec):
    Ej=Ejf(f,Ec)
    I=Ej*2*np.pi/F0*h*1e18
    return I
def Ejf(f,Ec):
    Ej=(f+Ec)**2/(8*Ec)
    return Ej
def Eji(I):
    return I*F0/(2*np.pi)/1e9/h/1e9
def f(Ej,Ec):
    return np.sqrt(8*Ej*Ec)-Ec
def Ec(C):
    return e**2/(2*C)/h/1e9

In [10]:
j=0.5e-3

In [11]:
Ec(Cq)

0.15381529540446717

In [12]:
# f1=4.5
# f2=4.7
fmin=4.1
fmax=4.8
E_c=Ec(Cq)
E_c

0.15381529540446717

In [13]:
Ejf(fmin,E_c),Ejf(fmax,E_c)

(14.705092006483468, 19.942982520409455)

In [14]:
(I(fmin,E_c)+I(fmax,E_c))/2,(I(fmax,E_c)-I(fmin,E_c))/2

(34.8799994010522, 5.272951541555663)

In [15]:
(I(fmin,E_c)+I(fmax,E_c))/2/j/250,(I(fmax,E_c)-I(fmin,E_c))/2/j/100

(279.03999520841757, 105.45903083111327)

### Real critical current

In [16]:
# S2=250*257
# S1=100*97
S2=250*279
S1=100*105

In [17]:
# I1=j*S1
# I2=j*S2
I1=j*S1
I2=j*S2
I1,I2

(5.25, 34.875)

In [18]:
f(Eji(I2-I1),Ec(Cq)),f(Eji(I1+I2),Ec(Cq))

(4.101289448340075, 4.7982754952161235)

### coupling between qubits

In [22]:
fq1=4.1e9
fq2=fq1
Ccq1=0.2842e-15*e_eff# for d=5mkm
Ccq2=0.101e-15*e_eff# for d=30mkm
Cq1=Cq
Cq2=Cq1
g1=2*np.pi*Ccq1/np.sqrt(Cq1*Cq2)*np.sqrt(fq1*fq2)
g2=2*np.pi*Ccq2/np.sqrt(Cq1*Cq2)*np.sqrt(fq1*fq2)
g1/Mhz,g2/Mhz

(57.59861591695502, 20.4695996045477)

### Qubit-drive relaxation

In [56]:
Cc=0.0056716e-15*e_eff
Cc=60e-18
Cq=81e-15
w=5e9# in Ghz
Z=50
# R=1/((w*Cc)**2)/Z
T1=(Cc+Cq)/(Z*(w*w*Cc*Cc)*np.pi)
T1*1e6#in mcs

5733.822083124015

## For two-qubit gates
everything you should know about simple two-qubit gates you can find in Skriptbuch 2013 

In [28]:
# everything in megahertz
w_1=6.62e3
w_2=6.51e3
w_c=7.026e3
g_q_c=70
delta=0.265*1e3
delta12=np.abs(w_2-w_1)
#coupling through a quantum bus two-photon virtual coupling rate J00 page(305)
J=(g_q_c**2/2)*(1/(w_c-w_1)+1/(w_c-w_2))
## for ISWAP gatw we start with the qubit veru far apart to provide J<<delta12 interaction is off
## tune them into resonance and obtain |01>+<10| with the angle=J*time and angle=pi/4 for sqrt(ISWAP)and pi/2 for ISWAP
zz=-2*(2*delta)/((delta12-delta)*(delta12+delta))*J**2 ## here J is a direct coupling or through a quantum bus 
## zz = E11-E10-E01 angle=J*time and angle=pi for CZ
## The higher level does not have to be used in an adiabatic way, instead we could suddenly
##tune the qubits into the point delta=+-delta12 and |11> will swap with |02> and |02> (like ISWAP) and when it comes back
## we will obtain an additional phase -1 page(309)
zx = (J/delta12)*delta/(delta-delta12) ## they said that we can make ZZ very low just by placing them far a part, 
##or we can use additional echo pulse scheme to refocuse this interaction page(311-313)
## delta>>delta12 to get ZX with small IX rotation they differ by factor delta12/delta
## this gate has complex calibration and  amplitude dependence 
zx_amp=10##in mhz already. also we should multiply zx on drive amplitude(Raby_freq)
J,zx*zx_amp,zz

(10.782544774124567, 1.6758793930457538, 2.120242032075029)

### For Q

In [4]:
import numpy as np
c=299792458
vl=c/np.sqrt(e_eff)
fr=7.2e9
lc=0.5/1e3
lo=0.689/1e3
x=1362/1e6
psi=2*np.pi*(lc+2*lo)*fr/vl
theta=2*np.pi*lc*fr/vl
omega=2*np.pi*fr/vl*x
k=(1+np.cos(psi+theta)-np.cos(theta-psi)-np.cos(2*omega-psi+theta))
k, 4400

-0.11745053409967654

In [18]:
import numpy as np
c=299792458
vl=c/np.sqrt(e_eff)
fr=7.4e9
lc=0.45/1e3
lo=0.689/1e3
x=1366/1e6
psi=2*np.pi*(lc+2*lo)*fr/vl
theta=2*np.pi*lc*fr/vl
omega=2*np.pi*fr/vl*x
k=(1+np.cos(psi+theta)-np.cos(theta-psi)-np.cos(2*omega-psi+theta))
k, 4400

(-0.09102193575496298, 4400)

In [19]:
np.cos(psi+theta)

0.6360069029655796

In [20]:
np.cos(theta-psi)

0.8611726922384668

In [21]:
np.cos(2*omega-psi+theta)

0.8658561464820759